## <span style='color:#ff5f27'> 📝 Colab Users - Uncomment & Run the following 2 Cells

In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")

Local environment
Added the following directory to the PYTHONPATH: f:\Assignment\ID2223 Scalable Machine Learning and Deep Learning\lab1\mlfs-book


## <span style='color:#ff5f27'> 📝 Imports

In [2]:
from xgboost import XGBRegressor
import hopsworks
from openai import OpenAI
from mlfs.airquality.llm_chain import (
    load_model, 
    get_llm_chain, 
    generate_response, 
    generate_response_openai,
)
import pandas as pd
import os
import warnings
from mlfs import config
warnings.filterwarnings("ignore")

2025-11-12 16:05:53,408 WARNING: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute

2025-11-12 16:05:53,408 WARNING: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute



## <span style="color:#ff5f27;"> 🔮 Connect to Hopsworks Feature Store </span>

In [3]:
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")
project = hopsworks.login()
fs = project.get_feature_store() 

HopsworksSettings initialized!
2025-11-12 16:05:56,568 INFO: Initializing external client


2025-11-12 16:05:56,568 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-12 16:05:58,378 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1294484


In [4]:
# Get_or_create the 'air_quality_fv' feature view
feature_view = fs.get_feature_view(
    name='air_quality_fv',
    version=1
)

# Initialize batch scoring
feature_view.init_batch_scoring(1)

weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

## <span style="color:#ff5f27;">🪝 Retrieve AirQuality Model from Model Registry</span>

In [5]:
# Retrieve the model registry
mr = project.get_model_registry()

# Retrieve the 'air_quality_xgboost_model' from the model registry
retrieved_model = mr.get_model(
    name="air_quality_xgboost_model",
    version=1,
)

# Download the saved model artifacts  to a local directory
saved_model_dir = retrieved_model.download()

Downloading: 0.000%|          | 0/551252 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/110345 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/19773 elapsed<00:00 remaining<?

In [6]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# model_air_quality = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
model_air_quality = XGBRegressor()

model_air_quality.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
model_air_quality

XGBRegressor(base_score='7.467858E1', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['float', 'float', 'float', 'float'], gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## <span style='color:#ff5f27'>⬇️ LLM Loading

In [7]:
import time
start_time = time.time()

# Load the LLM and its corresponding tokenizer.
model_llm, tokenizer = load_model(model_id="imiraoui/OpenHermes-2.5-Mistral-7B-sharded")

duration = time.time() - start_time
print(f"The code execution took {duration} seconds.")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
W1112 16:06:45.470000 30380 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

## <span style='color:#ff5f27'>⛓️ LangChain

In [ ]:
import time
start_time = time.time()


# Create and configure a language model chain.
llm_chain = get_llm_chain(
    model_llm,
    tokenizer,
)

duration = time.time() - start_time
print(f"The code execution took {duration} seconds.")

## <span style='color:#ff5f27'>🧬 Domain-specific Evaluation Harness

**Systematic evaluations** that can run automatically in CI/CD pipelines are key to evaluating models/RAG. 


In [ ]:
QUESTION7 = "Hi!"

response7 = generate_response(
    QUESTION7,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response7)

In [ ]:
QUESTION = "Who are you?"

response = generate_response(
    QUESTION,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
QUESTION1 = "What was the average air quality from 2024-01-10 till 2024-01-14?"

response1 = generate_response(
    QUESTION1, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response1)

In [ ]:
QUESTION11 = "When and what was the air quality like last week?"

response11 = generate_response(
    QUESTION11, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response11)

In [ ]:
QUESTION12 = "When and what was the minimum air quality from 2024-01-10 till 2024-01-14?"

response12 = generate_response(
    QUESTION12, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response12)

In [ ]:
QUESTION2a = "What was the air quality like last week?"

response2 = generate_response(
    QUESTION2a,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response2)

In [ ]:
QUESTION2 = "What was the air quality like yesterday?"

response2 = generate_response(
    QUESTION2,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response2)

In [ ]:
QUESTION3 = "What will the air quality be like next Tuesday?"

response3 = generate_response(
    QUESTION3, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response3)

In [ ]:
QUESTION4 = "What will the air quality be like the day after tomorrow?"

response4 = generate_response(
    QUESTION4, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response4)

In [ ]:
QUESTION5 = "What will the air quality be like this Sunday?"

response5 = generate_response(
    QUESTION5, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response5)

In [ ]:
QUESTION7 = "What will the air quality be like for the rest of the week?"

response7 = generate_response(
    QUESTION7, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response7)

In [ ]:
QUESTION = "Will the air quality be safe or not for the next week?"

response = generate_response(
    QUESTION7, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
QUESTION = "Is tomorrow's air quality level dangerous?"

response = generate_response(
    QUESTION, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
QUESTION = "Can you please explain different PM2_5 air quality levels?"

response = generate_response(
    QUESTION, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# !pip install openai --quiet
# !pip install gradio==3.40.1 --quiet

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np
from xgboost import XGBRegressor
from functions.llm_chain import load_model, get_llm_chain, generate_response


In [ ]:
# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio
    y = y.astype(np.float32)
    if y.ndim > 1 and y.shape[1] > 1:
        y = np.mean(y, axis=1)
    y /= np.max(np.abs(y))
    return transcriber({"sampling_rate": sr, "raw": y})["text"]

def generate_query_response(user_query, method, openai_api_key=None):
    if method == 'Hermes LLM':        
        response = generate_response(
            user_query,
            feature_view,
            weather_fg,
            model_air_quality,
            model_llm,
            tokenizer,
            llm_chain,
            verbose=False,
        )
        return response
    
    elif method == 'OpenAI API' and openai_api_key:
        client = OpenAI(
            api_key=openai_api_key
        )
        
        response = generate_response_openai(   
            user_query,
            feature_view,
            weather_fg,
            model_air_quality,
            client=client,
            verbose=True,
        )
        return response
        
    else:
        return "Invalid method or missing API key."

def handle_input(text_input=None, audio_input=None, method='Hermes LLM', openai_api_key=""):
    if audio_input is not None:
        user_query = transcribe(audio_input)
    else:
        user_query = text_input
    
    # Check if OpenAI API key is required but not provided
    if method == 'OpenAI API' and not openai_api_key.strip():
        return "OpenAI API key is required for this method."

    if user_query:
        return generate_query_response(user_query, method, openai_api_key)
    else:
        return "Please provide input either via text or voice."
    

# Setting up the Gradio Interface
iface = gr.Interface(
    fn=handle_input,
    inputs=[
        gr.Textbox(placeholder="Type here or use voice input..."), 
        gr.Audio(), 
        gr.Radio(["Hermes LLM", "OpenAI API"], label="Choose the response generation method"),
        gr.Textbox(label="Enter your OpenAI API key (only if you selected OpenAI API):", type="password")  # Removed `optional=True`
    ],
    outputs="text",
    title="🌤️ AirQuality AI Assistant 💬",
    description="Ask your questions about air quality or use your voice to interact. Select the response generation method and provide an OpenAI API key if necessary."
)

iface.launch(share=True)


---